In this notebook we first do the classification using the transformer This is our first classification task.

The output classification vector from the transformer is saved to be used by the FCNN This is our second classification task.

In [1]:
# Importing necessary libraries
import pandas as pd
from datetime import datetime
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [130]:
from simpletransformers.classification import ClassificationModel

In [131]:
input1 = torch.randn(1,4)
input2 = torch.randn(1,4)
output = torch.cosine_similarity(input1, input2)
print(input1)
print(input2)

print(output)

tensor([[-0.6092,  0.9993,  0.0077, -2.5364]])
tensor([[ 1.3585,  0.1174, -0.8305, -1.0718]])
tensor([0.3727])


## Preparing the dataset

Some pre-processing to the dataset has already been done in preparation for various tests, so this processing is not from scratch.

In [132]:
# procedure for getting the data sets and formatting them for the transformer
 

def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','Truths','Context'
],axis=1)
     
    return ReadSet


In [133]:
# preparing the training dataset
train=prepareDataset( 'train.xlsx')
# and display for inspecting
train

,text,labels
0,The attorney general requires that rape victim...,0
1,President Clinton reduced the scale of our mil...,3
2,"I used tax cuts to help create over 80,000 job...",4
3,"New Mexico moved ""up to"" sixth in the nation i...",4
4,"Corporate profits are up, CEO pay is up, but a...",5
...,...,...
10264,"Under Obamacare, premiums have doubled and tri...",4
10265,We adopted the modern Social Security system a...,5
10266,More than two months ago President Barack Obam...,3
10267,"We had a massive landslide victory, as you kno...",1


In [134]:
# preparing the evaluation/validation dataset
Eval=prepareDataset('valid.xlsx')
# and display for inspecting
Eval

,text,labels
0,The president is brain-dead.,0
1,"Barack Obama supported keeping troops in Iraq,...",3
2,"He's leading by example, refusing contribution...",3
3,I'm the first person who really took up the is...,4
4,I built that border fence in San Diego...and i...,4
...,...,...
1279,CNN accidentally aired 30 minutes of pornograp...,1
1280,President Obamas American Recovery and Reinves...,2
1281,We (in Illinois) have the fifth-highest tax bu...,4
1282,Says Donald Trump won more counties than any c...,4


In [135]:
# preparing the test set dataset
test=prepareDataset('test.xlsx')
test

,text,labels
0,New Mexico was 46th in teacher pay (when he wa...,4
1,Barack Obama and Hillary Clinton have changed ...,3
2,I'll tell you what I can tell this country: If...,1
3,Tommy Thompson created the first school choice...,5
4,Fifty-six percent decline in overall crime. A ...,5
...,...,...
1278,"We have trade agreements with 20 countries, an...",1
1279,On Donald Trumps plan to cut federal funding t...,4
1280,"Black Lives Matter, who are attacking law enfo...",2
1281,Latina who enthusiastically supported Donald T...,0


## Setting up the transformer for fine tuning

This is where changes are done to optimise the model

The simpletransformers library is the quickest way to do this at the time of writing. 
For more information on the settings and their default value go here:
https://github.com/ThilinaRajapakse/simpletransformers#default-settings 

###### Please do read that reference before changing any parameters. Don't try to be a hero!

In [139]:
#Set the model being used here
model_class='albert'  # bert or roberta or albert
model_version='albert-large-v2' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2


output_folder='./TunedModels/'+model_class+'/'+model_version+"/"
cache_directory= "./TunedModels/"+model_class+"/"+model_version+"/cache/"
labels_count=6  # the number of classification classes

print('model variables were set up: ')

model variables were set up: 


In [140]:
# use this to test if writing to the directories is working

import os
print(os.getcwd())
print(output_folder)
print(cache_directory)

testWrite=train.head(30)
 
testWrite.to_csv(output_folder+'DeleteThisToo.tsv', sep='\t')
testWrite.to_csv(cache_directory+'DeleteThisToo.tsv', sep='\t')

del(testWrite)

G:\0 finalThesis\siamese
./TunedModels/albert/albert-large-v2/
./TunedModels/albert/albert-large-v2/cache/


In [146]:
 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    "save_steps": save_every_steps, 
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 8,
    "evaluate_during_training_steps": 312,
    "max_seq_length": 64,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

# You can set class weights by using the optional weight argument

### Loading a saved model (based on above args{})

If you stopped training you can continue training from a previously saved check point.
The next cell allows you to load a model from any checkpoint.
The number of epochs in the train_args{} will be done and continue tuning from your checkpoint.

###### HOWEVER
It will overwrite previous checkpoints!
Example:  If you load an epoch-3 checkpoint, the epoch-1 checkpoint will be overwritten by the 4th epoch and it will be equivalent to a 4th epoch even if you have epoch-1 in the name.
###### SO BE CAREFUL

In [150]:
# loading a previously saved model based on this particular Transformer Class and model_name

# loading the checkpoint that gave the best result
CheckPoint='checkpoint-161-epoch-3'  #epoch 1


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )

Loading model, please wait...
model in use is : ./TunedModels/albert/albert-large-v2/checkpoint-161-epoch-3


## Training the Transformer

Skip the next cell if you want to skip the training and go directly to the evaluation

In [151]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.753227Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.537029Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.507523

Training of albert model complete. Saved to ./TunedModels/albert/albert-large-v2/.
Training time:  0:03:08.390604


## Evaluating the training

In [152]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)

EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Features loaded from cache at ./TunedModels/albert/albert-large-v2/cache/cached_dev_albert_64_6_10269



{'mcc': 0.20466921249805126, 'acc': 0.35018015386113543, 'eval_loss': 1.558491243937305}
Features loaded from cache at ./TunedModels/albert/albert-large-v2/cache/cached_dev_albert_64_6_1284



{'mcc': 0.09416109138513852, 'acc': 0.26557632398753894, 'eval_loss': 1.6665791613715035}
Features loaded from cache at ./TunedModels/albert/albert-large-v2/cache/cached_dev_albert_64_6_1283



{'mcc': 0.11524783095303312, 'acc': 0.2821512081060016, 'eval_loss': 1.6612607847829783}
Training Result: 0.35018015386113543
Eval Result: 0.26557632398753894
Test Set Result: 0.2821512081060016


In [153]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-1.7587891  -0.18713379 -0.25805664  0.7626953   0.9614258   0.9638672 ] 5   4 
[-0.07824707  0.62353516  0.3293457   0.11523438 -0.49829102 -0.60498047] 1   3 
[ 0.11871338  0.39746094  0.09368896  0.10760498 -0.3293457  -0.46264648] 1   1 Match 1

[-1.3369141  -0.3317871   0.15209961  1.0771484   0.8388672   0.16247559] 3   5 
[-1.6191406  -0.5517578  -0.15795898  1.1708984   0.7915039   0.2397461 ] 3   5 
[-1.671875   -0.24450684 -0.12158203  0.89501953  0.93603516  0.81347656] 4   2 
[-1.7392578  -0.41430664 -0.38916016  1.0058594   1.0585938   0.7133789 ] 4   4 Match 2

[-1.4775391  -0.17382812  0.07879639  0.68847656  0.73046875  0.44873047] 4   5 
[-1.7880859  -0.23486328 -0.39086914  0.6459961   1.2226562   1.1923828 ] 4   4 Match 3

[-1.5947266  -0.22485352 -0.19970703  0.8613281   1.0185547   0.60302734] 4   5 
[-0.4194336   0.25268555  0.4345703   0.5605469  -0.1472168  -0.8354492 ] 3   5 
[-1.6865234   0.00200653  0.06481934  0.8300781   0.84277344  0.4716797 ] 4   3 
[-0.

[-1.7001953  -0.08178711  0.14575195  1.0048828   0.8105469   0.2487793 ] 3   2 
[ 0.0447998   0.84033203 -0.05731201 -0.20080566 -0.6699219  -0.44067383] 1   2 
[ 0.3840332   0.5961914   0.22924805 -0.16699219 -0.69189453 -0.6479492 ] 1   1 Match 33

[-0.41552734  0.51708984  0.07446289  0.15307617 -0.16552734 -0.12329102] 1   2 
[-1.5361328   0.06719971  0.3930664   0.59228516  0.7026367  -0.00733566] 4   1 
[-0.49121094  0.46801758  0.28149414  0.4819336  -0.14526367 -0.48950195] 3   2 
[-1.6689453  -0.07092285 -0.5053711   0.5942383   0.9916992   1.0449219 ] 5   5 Match 34

[-0.8623047   0.6245117  -0.15368652 -0.01268768  0.15576172  0.44335938] 1   5 
[-1.8144531  -0.11853027 -0.29467773  0.6923828   0.99658203  1.0429688 ] 5   2 
[-0.73535156  0.14001465  0.42016602  0.80322266 -0.03515625 -0.68847656] 3   3 Match 35

[ 0.03643799  0.3684082   0.7685547   0.56591797 -0.69970703 -1.3632812 ] 2   1 
[-1.796875   -0.29345703 -0.44335938  0.9140625   1.1972656   1.1298828 ] 4   2 
[

[-1.6357422  -0.40307617 -0.10455322  1.0175781   1.2880859   0.86279297] 4   4 Match 67

[-1.1503906   0.4033203   0.32836914  0.52197266  0.49853516  0.13928223] 3   1 
[-0.5991211   0.28881836  0.18481445  0.5107422  -0.0871582  -0.23352051] 3   1 
[-1.6650391   0.1229248  -0.06158447  0.79003906  0.9086914   0.5839844 ] 4   1 
[-0.07580566  0.96728516 -0.08312988 -0.31811523 -0.5053711  -0.26782227] 1   0 
[-0.9716797   0.45166016  0.35791016  0.4970703  -0.03019714 -0.5541992 ] 3   4 
[-0.3798828   0.29052734  0.4050293   0.5957031  -0.41796875 -0.6958008 ] 3   1 
[-1.3466797  -0.0068779   0.3088379   0.96435547  0.42773438 -0.32958984] 3   3 Match 68

[ 0.08807373  0.49682617  0.4104004  -0.12133789 -0.3996582  -0.4189453 ] 1   1 Match 69

[-0.6933594   0.25830078  0.07885742  0.1796875  -0.13708496  0.11566162] 1   1 Match 70

[-1.7734375   0.04467773 -0.42236328  0.62353516  1.0664062   1.0576172 ] 4   3 
[-0.35009766  0.4453125   0.35742188  0.02372742 -0.20349121 -0.25463867]

[-1.1494141   0.21838379  0.38354492  0.8901367   0.32080078 -0.41430664] 3   1 
[-1.6533203  -0.48046875 -0.29125977  0.8959961   1.2158203   0.92529297] 4   3 
[-1.7646484  -0.29223633 -0.30786133  0.8442383   1.0810547   0.81396484] 4   2 
[-1.6962891   0.0585022  -0.11743164  0.65234375  1.0214844   0.5571289 ] 4   2 
[ 0.00727844  0.48120117  0.12463379  0.18737793 -0.2607422  -0.4560547 ] 1   1 Match 111

[-1.6298828  -0.5185547   0.08239746  1.0029297   0.88427734  0.54248047] 3   4 
[-1.4677734   0.3942871  -0.27905273  0.27001953  0.91259766  0.84521484] 4   1 
[-1.2783203   0.55078125  0.45654297  0.828125    0.06768799 -0.7524414 ] 3   4 
[-1.7666016  -0.20227051 -0.06018066  0.8647461   0.91503906  0.5644531 ] 4   3 
[-0.47216797  0.77246094  0.07055664 -0.00107384 -0.2277832  -0.00348663] 1   3 
[-1.7568359  -0.03515625 -0.06872559  0.79296875  1.0126953   0.46606445] 4   3 
[-1.390625   -0.1973877   0.11303711  1.1367188   0.73095703  0.14428711] 3   3 Match 112

[-1.0224

[ 0.30004883  0.84521484  0.04595947 -0.4013672  -0.7910156  -0.46679688] 1   3 
[-1.3808594  -0.05239868  0.74560547  1.1005859   0.66845703 -0.5180664 ] 3   5 
[-0.28100586  0.7060547   0.4946289   0.22668457 -0.3071289  -0.77001953] 1   1 Match 154

[ 0.31445312  0.7788086   0.27661133 -0.08410645 -0.7290039  -0.7915039 ] 1   3 
[-1.8691406  -0.2355957  -0.28320312  0.7817383   1.1972656   0.9824219 ] 4   5 
[-1.1787109   0.2142334  -0.08850098  0.35961914  0.44482422  0.6958008 ] 5   3 
[-0.78808594  0.7607422   0.47021484  0.4453125  -0.17297363 -0.85302734] 1   1 Match 155

[-1.6865234  -0.10345459 -0.21533203  0.63378906  1.1484375   0.8745117 ] 4   4 Match 156

[-1.7353516   0.02287292 -0.2166748   0.59521484  0.9511719   0.84277344] 4   3 
[-1.5712891   0.05020142  0.21704102  0.94873047  0.6738281   0.11535645] 3   3 Match 157

[-0.79052734  0.44018555  0.28320312  0.81152344 -0.12011719 -0.8808594 ] 3   3 Match 158

[-1.7324219  -0.35742188 -0.46777344  0.79052734  1.2802734

...

[-1.1162109   0.40551758  0.1307373   0.39697266  0.40014648  0.078125  ] 1   4 
[-1.2832031   0.39819336 -0.3251953   0.2409668   0.37182617  0.8808594 ] 5   4 
[-0.22070312  0.5498047   0.40820312  0.31933594 -0.4177246  -0.9121094 ] 1   3 
[-1.4863281   0.14526367  0.45751953  1.0615234   0.45263672 -0.23913574] 3   2 
[-0.21594238  0.29760742  0.2619629   0.21350098 -0.0406189  -0.11126709] 1   5 
[-1.4482422   0.08325195 -0.10723877  0.609375    0.8461914   0.45410156] 4   5 
[-0.5961914   0.6386719  -0.16296387  0.01985168 -0.04605103  0.25439453] 1   1 Match 223

[-1.4101562   0.32006836  0.04788208  0.63720703  0.61376953  0.18334961] 3   1 
[-1.6103516   0.31176758 -0.20178223  0.30859375  0.7631836   0.7368164 ] 4   3 
[-1.7724609  -0.03881836 -0.36767578  0.53808594  1.1669922   0.91259766] 4   2 
[-1.8486328  -0.09692383 -0.49853516  0.53808594  1.2070312   1.0849609 ] 4   5 
[-0.5654297   0.55126953  0.25146484  0.42651367 -0.03311157 -0.4885254 ] 1   2 
[-0.6220703   0.47


[-0.7919922   0.30371094  0.4897461   0.7368164   0.10314941 -0.82421875] 3   4 
[-1.5742188   0.08978271 -0.234375    0.39916992  0.7006836   0.8354492 ] 5   4 
[-0.87402344  0.34204102  0.1977539   0.14196777  0.0094986   0.25854492] 1   5 
[-0.22619629  0.72509766 -0.03201294 -0.10919189 -0.38232422  0.13916016] 1   3 
[-0.43530273  0.5913086   0.43164062  0.26757812 -0.20117188 -0.6479492 ] 1   2 
[ 0.890625    0.5957031   0.22167969 -0.33862305 -0.96777344 -0.8183594 ] 0   0 Match 257

[-1.4736328   0.074646    0.24841309  0.7036133   0.57128906  0.38452148] 3   4 
[-1.4316406  -0.02362061  0.25317383  1.015625    0.69970703 -0.14123535] 3   2 
[-0.56591797  0.10516357  0.32958984  0.29589844 -0.02992249 -0.0526123 ] 2   2 Match 258

[-1.3056641   0.50878906  0.2890625   0.58447266  0.44335938  0.18835449] 3   2 
[-1.5498047  -0.36865234  0.26000977  1.0263672   1.0576172   0.18054199] 4   4 Match 259

[-0.19189453  0.3317871   0.51464844  0.34033203 -0.33813477 -0.8256836 ] 2   

[-0.34106445  0.59521484  0.09753418 -0.01676941 -0.29833984  0.01161957] 1   1 Match 297

[-0.33032227  0.36376953  0.35229492  0.140625   -0.17089844 -0.22058105] 1   1 Match 298

[-0.70166016  0.13122559  0.7182617   0.58691406  0.090271   -0.7631836 ] 2   3 
[-1.2304688   0.51220703  0.34350586  0.6328125  -0.02798462 -0.6040039 ] 3   0 
[-1.6123047   0.10900879 -0.24938965  0.64746094  1.0566406   0.68408203] 4   1 
[-1.46875    -0.06237793 -0.4855957   0.3840332   1.0527344   0.7685547 ] 4   5 
[-0.8598633   0.36694336  0.5708008   0.5961914   0.08300781 -0.32958984] 3   2 
[-0.3959961   1.0625     -0.25048828 -0.1072998  -0.4645996  -0.26049805] 1   3 
[-0.7133789   0.25512695  0.21008301  0.15429688 -0.04989624  0.29589844] 5   4 
[-0.50683594  0.6298828   0.43847656  0.19311523  0.04266357 -0.58740234] 1   2 
[-0.31079102  0.8515625   0.36938477  0.28857422 -0.3918457  -0.7841797 ] 1   1 Match 299

[-1.3916016   0.33691406 -0.15283203  0.32250977  0.5678711   0.6816406 ] 5   3

[-0.96484375  0.3395996  -0.03305054  0.21484375  0.35327148  0.07873535] 4   4 Match 329

[-0.7207031   1.0869141  -0.38745117 -0.10748291 -0.2998047  -0.21520996] 1   3 
[ 0.39282227  0.7285156   0.30639648 -0.21728516 -0.7866211  -0.78027344] 1   3 
[-1.4394531   0.17126465 -0.13879395  0.5336914   0.3630371   0.78222656] 5   4 
[-1.7841797  -0.05825806 -0.42700195  0.3149414   1.2177734   1.0107422 ] 4   5 
[-0.09637451  0.69189453  0.21130371 -0.20788574 -0.4519043  -0.28710938] 1   5 
[ 0.70458984  0.59765625  0.02883911 -0.5644531  -0.96875    -0.60253906] 0   5 
[-0.9326172   0.6123047  -0.11621094  0.38110352  0.25439453  0.09747314] 1   2 
[-1.8076172  -0.34545898 -0.37939453  0.890625    1.2578125   1.0097656 ] 4   4 Match 330

[-1.6748047   0.02682495 -0.33398438  0.53222656  0.92041016  1.0498047 ] 5   2 
[-1.0449219   0.03262329  0.48364258  0.5546875   0.1907959  -0.28393555] 3   4 
[-0.5786133   0.54003906  0.38598633  0.3972168  -0.02575684 -0.47485352] 1   4 
[ 0.3361

[-1.3691406   0.3112793  -0.00666809  0.42407227  0.5810547   0.52197266] 4   2 
[-0.7026367   0.47753906  0.01100922 -0.03244019 -0.09014893  0.21594238] 1   2 
[-0.43554688  0.3322754   0.35839844  0.3840332   0.00083256 -0.4711914 ] 3   3 Match 358

[-0.25024414  0.8623047   0.22314453 -0.00201035 -0.54345703 -0.7265625 ] 1   5 
[-0.07122803  0.8305664   0.20117188 -0.21313477 -0.5673828  -0.4182129 ] 1   4 
[-1.1884766   0.1126709   0.58984375  0.88720703  0.25878906 -0.5708008 ] 3   2 
[-1.5791016  -0.08703613  0.12103271  0.9243164   0.8251953   0.44433594] 3   3 Match 359

[ 0.07250977  0.8666992  -0.06240845 -0.4777832  -0.63671875 -0.11029053] 1   3 
[-0.34228516  0.13513184  0.28051758  0.16723633 -0.28466797 -0.14453125] 2   4 
[-1.6289062   0.14465332 -0.17089844  0.81640625  0.6113281   0.45898438] 3   4 
[-0.9916992   0.01139069  0.43798828  0.6333008   0.3581543  -0.02700806] 3   2 
[-1.7470703  -0.18493652 -0.40356445  0.4206543   1.3105469   1.015625  ] 4   3 
[-1.7792

In [154]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[ 11  43   8  17  11   2]
 [  8 113  13  57  52   7]
 [  4  73  23  66  39   9]
 [  2  73  21  85  79   7]
 [  0  45   8  66 113  17]
 [  2  47   8  44  93  17]]


In [156]:
target_names = ['Pants', 'False', 'Barely-True','Hlaf-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.41      0.12      0.18        92
       False       0.29      0.45      0.35       250
 Barely-True       0.28      0.11      0.16       214
   Hlaf-True       0.25      0.32      0.28       267
 Mostly-True       0.29      0.45      0.36       249
        True       0.29      0.08      0.13       211

    accuracy                           0.28      1283
   macro avg       0.30      0.26      0.24      1283
weighted avg       0.29      0.28      0.26      1283



In [157]:
metrics.accuracy_score(test['labels'],Pred)

0.2821512081060016

In [158]:
# saving the output of the models to CSVs
#these are 1X6 classification vectors

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
print('Saving...')
trainOut = pd.DataFrame(data= TrainModel_outputs )
trainOut.to_csv(SavesDirectory+'trainOut.tsv', sep='\t',  index=False)

evalOut = pd.DataFrame(data= EvalModel_outputs )
evalOut.to_csv(SavesDirectory+'evalOut.tsv', sep='\t',  index=False)

testOut = pd.DataFrame(data= TestModel_outputs )
testOut.to_csv(SavesDirectory+'testOut.tsv', sep='\t',  index=False)

print('Saving Complete on',datetime.now() ,'in:', SavesDirectory)

Saving...
Saving Complete on 2020-03-28 19:26:48.683056 in: ./TunedModels/albert/albert-large-v2/Saves/


In [159]:
del(model)
del(train,Eval,test)
del(trainOut,evalOut,testOut)
torch.cuda.empty_cache()

#  Adding the reputation vector

This section takes the output results from the transformer used above and uses it together with the speaker's reputation to enhance the classification.

Before running this section it is suggested that you halt the program and start running it again from this cell. The neural net will likely have an error caused by some unreleased variable used by thr simple transformers library. 

In [1]:
import pandas as pd
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:

train=pd.read_excel('trainReputation.xlsx' )
train=train.iloc[:,:-2].astype(float)
train=train/200  #for scaling
#train

model_class='albert'  # bert or roberta or albert
model_version='albert-large-V2' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'trainOut.tsv', sep='\t')

train=pd.concat([train,TF_Output], axis=1)

train

,PantsTotal,NotRealTotal,BarelyTotal,HalfTotal,MostlyTotal,Truths,0,1,2,3,4,5
0,0.005,0.000,0.000,0.000,0.000,0.000,-0.042389,0.749023,-0.128662,-0.281494,-0.387207,0.003132
1,0.095,0.160,0.170,0.290,0.165,0.165,-1.463867,-0.130737,0.175659,0.963379,0.816895,0.170044
2,0.005,0.010,0.005,0.015,0.040,0.010,-1.208984,0.086731,0.572754,1.009766,0.365723,-0.464844
3,0.005,0.010,0.005,0.015,0.040,0.010,-1.789062,-0.198242,-0.172974,0.847656,1.080078,0.759277
4,0.035,0.145,0.200,0.345,0.380,0.365,-1.682617,-0.299805,-0.057770,1.031250,1.160156,0.654785
...,...,...,...,...,...,...,...,...,...,...,...,...
10264,0.005,0.030,0.070,0.050,0.050,0.020,-1.038086,0.406006,0.380127,0.839355,0.057983,-0.520996
10265,0.055,0.075,0.080,0.100,0.050,0.035,-1.597656,-0.008057,0.088074,0.793457,0.835938,0.486328
10266,0.035,0.115,0.140,0.190,0.170,0.075,-1.671875,-0.160645,0.019196,0.981445,0.876465,0.415039
10267,0.305,0.570,0.315,0.255,0.185,0.070,-0.340088,0.747070,0.042603,-0.229980,-0.360107,-0.011665


In [3]:
TrainLables=pd.read_excel('trainReputation.xlsx' )
TrainLables=TrainLables.iloc[:,-1] 

TrainLables=pd.get_dummies(TrainLables)
TrainLables

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
10264,0,0,0,0,1,0
10265,0,0,0,0,0,1
10266,0,0,0,1,0,0
10267,0,1,0,0,0,0


In [4]:
input=torch.tensor(train.values)
 
input

tensor([[ 0.0050,  0.0000,  0.0000,  ..., -0.2815, -0.3872,  0.0031],
        [ 0.0950,  0.1600,  0.1700,  ...,  0.9634,  0.8169,  0.1700],
        [ 0.0050,  0.0100,  0.0050,  ...,  1.0098,  0.3657, -0.4648],
        ...,
        [ 0.0350,  0.1150,  0.1400,  ...,  0.9814,  0.8765,  0.4150],
        [ 0.3050,  0.5700,  0.3150,  ..., -0.2300, -0.3601, -0.0117],
        [ 0.0000,  0.0050,  0.0000,  ...,  0.6709, -0.1808, -1.0039]],
       dtype=torch.float64)

In [5]:
targets=torch.tensor(TrainLables.astype(float).values)

targets

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        ...,
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=torch.float64)

In [6]:
 
size= torch.tensor(input[0].size())
InputSize=size.item()

OutputSize=torch.tensor(targets[0].size()).item()

print('input size:', InputSize)
print('output size:', OutputSize)

input size: 12
output size: 6


In [7]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
         
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(InputSize, 24)  # input size 32
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, OutputSize)  #classifies 'outputsize' different classes

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x)) 
        x = torch.tanh(self.fc3(x)).double()
        return x

    

#now we use it

net = Net()

In [8]:
# here we  setup the neural network parameters
# pick an optimizer (Simple Gradient Descent)

learning_rate = 9e-4
criterion = nn.MSELoss()  #computes the loss Function

import torch.optim as optim

# creating optimizer
#optimizer = optim.SGD(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [27]:
for epoch in range(1000):  
        
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input.float())

    loss = criterion(output, targets)
    print('Loss:', loss, ' at epoch:', epoch)

    loss.backward()  #backprop
    optimizer.step()    # Does the update

Loss: tensor(0.0965, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 0
Loss: tensor(0.0965, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 1
Loss: tensor(0.0965, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2
Loss: tensor(0.0965, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 3
Loss: tensor(0.0965, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 4
Loss: tensor(0.0965, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 5
Loss: tensor(0.0964, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 6
Loss: tensor(0.0964, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7
Loss: tensor(0.0964, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 8
Loss: tensor(0.0964, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 9
Loss: tensor(0.0964, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 10
Loss: tensor(0.0964, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 11
Loss: tensor(0

Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 105
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 106
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 107
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 108
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 109
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 110
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 111
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 112
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 113
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 114
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 115
Loss: tensor(0.0962, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 203
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 204
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 205
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 206
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 207
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 208
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 209
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 210
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 211
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 212
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 213
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 315
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 316
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 317
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 318
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 319
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 320
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 321
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 322
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 323
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 324
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 325
Loss: tensor(0.0957, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 423
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 424
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 425
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 426
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 427
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 428
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 429
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 430
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 431
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 432
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 433
Loss: tensor(0.0955, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

...

Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2445
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2446
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2447
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2448
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2449
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2450
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2451
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2452
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2453
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2454
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2455
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2569
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2570
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2571
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2572
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2573
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2574
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2575
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2576
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2577
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2578
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2579
Loss: tensor(0.0930, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2693
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2694
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2695
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2696
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2697
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2698
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2699
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2700
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2701
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2702
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2703
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2816
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2817
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2818
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2819
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2820
Loss: tensor(0.0929, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2821
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2822
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2823
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2824
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2825
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2826
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2940
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2941
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2942
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2943
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2944
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2945
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2946
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2947
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2948
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2949
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2950
Loss: tensor(0.0928, dtype=torch.float64, grad_fn=<MseLossBackwar

In [34]:
#save the FCNN model

stage='NNetwork/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
PATH = SavesDirectory+'Tanh_MSE_adam4856.pth'

torch.save(net.state_dict(), PATH)

# more on saving pytorch networks: https://pytorch.org/docs/stable/notes/serialization.html

In [27]:
#load previously saved FCNN model 

stage='NNetwork/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
PATH = SavesDirectory+'Tanh_MSE_adam4856.pth'

net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [28]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0
labels=pd.read_excel('trainReputation.xlsx' )

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(input)):
        outputs = net(input[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if TrainLables.iloc[row,result]==1: correct+=1
        
        Y.append(labels.iloc[row])
        Pred.append(result)
        
        print(result, end=' ')
        
    
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))

0 3 4 4 4 1 4 5 5 5 3 3 4 5 3 5 3 3 4 3 4 2 5 1 3 5 5 5 5 1 4 1 3 3 5 5 3 5 5 4 3 4 5 4 5 5 2 1 5 1 4 5 3 1 5 3 5 2 1 3 3 4 5 5 3 3 4 5 4 3 4 3 5 1 4 4 4 3 3 1 4 4 5 4 4 3 3 1 3 1 3 3 3 1 3 5 5 1 3 4 5 0 4 4 4 3 1 3 5 3 1 2 3 3 3 3 3 4 4 3 4 2 2 5 1 4 5 3 1 2 3 3 4 5 4 5 3 5 4 1 1 3 5 3 3 3 5 5 4 2 1 3 1 4 3 1 5 2 5 3 5 4 3 5 1 0 3 3 3 3 4 4 4 3 3 5 2 2 0 3 4 5 3 4 5 1 5 5 3 5 5 1 3 4 5 5 2 4 3 2 1 2 0 5 0 5 1 5 5 5 3 1 0 0 1 3 5 3 4 5 4 5 3 2 1 2 0 3 0 3 5 5 1 1 2 3 5 5 5 5 5 4 1 5 5 5 1 5 3 4 3 3 5 4 2 5 3 5 5 5 5 0 5 5 5 3 4 4 1 1 1 5 5 0 5 1 0 0 3 4 3 0 3 1 5 1 4 3 5 1 5 1 1 4 2 0 3 1 3 4 4 3 3 4 1 1 3 4 4 1 1 1 2 1 3 4 2 0 3 0 3 1 4 1 5 5 5 5 1 5 5 1 1 4 5 4 1 5 2 3 4 5 5 4 3 1 3 4 2 3 0 2 3 5 4 0 1 3 5 2 0 5 4 3 4 4 3 4 5 3 4 1 1 5 5 1 2 3 3 3 3 5 1 5 5 1 5 5 5 3 5 1 0 1 5 3 1 4 3 4 5 4 5 1 3 1 2 1 3 5 4 1 3 3 2 2 3 5 3 3 4 5 3 3 1 1 4 1 1 5 5 3 1 0 1 1 1 1 1 2 3 4 4 4 5 4 5 5 1 1 1 1 3 3 3 1 1 1 4 1 1 2 0 1 1 1 5 4 1 1 1 1 1 3 1 3 4 4 2 3 1 2 3 1 5 5 2 0 2 1 1 1 1 5 5 5 3 3 3 3 

 3 3 3 4 2 1 4 3 3 4 5 1 4 5 3 1 4 1 3 3 1 3 2 5 4 5 2 4 3 4 5 3 5 3 3 5 1 1 0 4 3 2 2 4 3 0 4 0 2 4 0 4 3 4 4 3 4 0 1 2 1 3 5 3 4 1 5 1 4 3 4 5 5 4 1 4 5 3 4 4 1 5 1 3 3 5 4 5 1 1 3 0 3 3 0 4 2 5 2 2 3 0 0 2 1 1 1 1 5 1 2 3 0 3 3 5 4 4 3 2 3 2 4 2 5 4 5 3 2 5 5 3 3 3 1 4 3 1 5 1 3 1 5 2 2 3 1 2 5 4 3 3 1 3 3 4 3 3 5 2 3 0 2 5 2 1 4 4 1 1 4 4 2 2 2 1 3 3 3 1 2 3 2 5 4 5 3 4 1 4 1 2 1 4 5 1 0 4 1 4 5 3 4 5 3 5 5 3 4 2 3 3 3 2 4 4 3 3 4 3 3 1 3 1 5 4 2 1 3 0 1 0 4 5 4 3 2 0 5 5 0 5 3 5 1 0 3 3 4 2 5 2 5 5 0 2 2 2 3 3 4 4 2 3 1 3 1 1 4 1 5 4 1 4 5 4 5 3 3 3 4 5 5 5 5 3 2 1 2 5 1 2 4 1 1 4 3 1 4 1 2 5 1 3 2 4 3 2 0 4 2 5 1 1 0 2 2 3 3 0 1 2 3 1 1 3 3 4 1 4 1 0 4 4 2 5 3 2 3 3 2 0 3 3 1 2 2 4 2 3 1 0 3 2 3 1 1 4 5 2 3 3 4 3 2 1 3 2 2 4 5 4 3 5 0 5 1 3 4 5 0 4 5 3 3 5 4 2 3 5 5 3 3 3 2 2 4 0 4 2 1 3 1 4 5 3 3 3 0 1 5 4 4 1 1 4 1 3 3 1 2 5 3 1 3 5 5 5 0 1 5 5 2 3 3 3 3 5 1 2 2 1 2 5 4 5 5 1 5 3 1 2 1 2 0 4 1 4 3 2 1 3 5 5 2 5 4 1 0 1 2 3 4 1 3 0 3 5 3 5 4 5 1 3 3 3 3 3 0 4 5 5 4 5 3 4 1 1 0 0

 4 4 1 0 4 4 4 4 2 4 3 2 1 3 2 4 0 1 4 3 3 4 1 4 5 1 1 5 1 2 1 0 2 5 0 1 1 1 1 2 1 3 1 5 1 1 4 1 0 0 3 3 2 1 0 3 2 1 3 4 2 4 4 1 4 0 3 0 4 1 4 1 3 5 4 4 3 1 1 2 3 4 4 1 1 1 3 5 4 3 1 2 4 3 2 1 4 2 1 2 5 4 1 5 1 4 0 2 5 1 4 1 2 0 5 4 2 2 3 4 4 4 1 1 3 4 1 4 4 3 4 4 4 3 3 0 1 5 5 3 1 5 1 2 4 1 0 5 4 1 4 2 3 3 3 2 5 2 1 3 4 1 2 1 5 0 5 5 2 1 2 1 4 1 1 4 1 3 5 4 3 4 3 0 1 4 4 1 4 3 3 4 5 1 4 2 5 2 2 0 1 1 3 2 3 4 5 1 5 4 0 4 4 1 2 1 5 0 4 4 5 1 4 4 5 4 1 4 1 4 4 5 4 2 1 3 5 2 3 5 5 4 4 5 5 5 1 4 3 1 4 5 2 4 1 0 1 2 4 0 5 0 3 1 4 4 4 1 4 1 5 4 0 4 4 4 1 3 3 5 0 4 1 4 5 1 4 1 1 3 5 5 4 4 1 4 2 2 4 1 4 4 0 4 1 3 1 4 1 2 0 4 4 0 2 1 1 4 1 4 0 2 4 3 5 2 1 3 5 4 5 5 1 3 4 4 1 4 0 0 1 5 1 3 3 1 4 1 3 1 1 2 1 4 1 2 1 5 4 5 4 4 2 1 1 3 1 3 1 3 4 3 1 5 3 4 4 1 1 3 1 1 3 1 3 2 3 2 3 3 1 3 2 4 2 4 0 1 1 1 4 3 1 1 1 2 3 3 3 4 1 4 4 5 2 3 4 4 4 0 2 0 4 3 0 2 0 1 4 2 1 2 5 3 3 3 1 5 0 0 4 3 1 1 3 1 2 2 4 4 3 2 4 2 1 1 4 5 2 1 2 1 4 5 0 4 4 1 3 4 4 2 4 3 2 2 1 5 5 4 3 1 0 4 5 1 3 2 5 3 4 5 0 4 1 3 1 3 1 1

In [11]:
# load the validation data

ValidData=pd.read_excel('validReputation.xlsx' )
ValidData=ValidData.iloc[:,:-2].astype(float)
ValidData=ValidData/200

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'evalOut.tsv', sep='\t')

ValidData=pd.concat([ValidData,TF_Output], axis=1)


ValidData=torch.tensor(ValidData.values)
ValidData

tensor([[ 0.0200,  0.0500,  0.0550,  ..., -0.7241, -1.0283, -0.5869],
        [ 0.0100,  0.0300,  0.0300,  ...,  0.9609,  0.2737, -0.0728],
        [ 0.0450,  0.3550,  0.3500,  ...,  0.2751, -0.1869, -0.5029],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5620,  1.2666,  1.1582],
        [ 0.0000,  0.0500,  0.0400,  ...,  0.5962,  1.1602,  1.1270],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.7217,  1.0645,  0.8315]],
       dtype=torch.float64)

In [29]:
labels=pd.read_excel('validReputation.xlsx' )

labels=labels.iloc[:,-1] 
labelsOneHot=pd.get_dummies(labels)
labelsOneHot

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
...,...,...,...,...,...,...
1279,0,1,0,0,0,0
1280,0,0,1,0,0,0
1281,0,0,0,0,1,0
1282,0,0,0,0,1,0


In [30]:
ValidLables =torch.tensor(labelsOneHot.values)
ValidLables

tensor([[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0],
        ...,
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0]], dtype=torch.uint8)

In [31]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(ValidData)):
        outputs = net(ValidData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if labelsOneHot.iloc[row,result]==1: correct+=1
        
        Y.append(labels.iloc[row])
        Pred.append(result)
        
        print(result, end=' ')
        
    
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))

0 3 2 5 4 2 5 5 4 1 5 3 3 5 5 0 1 5 0 3 4 5 1 1 3 3 5 5 3 2 5 4 1 1 0 5 3 4 4 0 0 4 1 0 5 3 4 1 4 4 4 1 0 1 5 1 5 3 5 5 3 5 1 1 1 3 1 5 1 1 1 1 5 0 2 5 2 1 3 3 1 2 4 1 4 4 3 4 1 1 1 2 3 1 4 1 2 1 0 1 0 3 1 4 5 1 0 1 4 1 3 1 4 1 1 1 1 5 1 1 1 1 5 0 3 2 1 3 5 5 4 3 5 4 4 3 3 2 3 3 3 4 4 1 3 3 1 3 1 4 1 1 1 1 5 2 5 2 3 2 3 5 3 4 5 4 1 5 2 3 3 3 1 2 1 4 5 4 2 3 1 5 5 2 3 5 4 5 5 4 0 5 3 4 4 5 2 3 4 5 3 4 3 2 4 2 4 4 3 2 2 1 1 1 1 3 3 3 2 1 4 2 1 3 3 4 2 1 2 3 3 2 2 2 1 2 2 2 1 4 3 5 2 3 3 4 3 4 5 3 2 4 1 1 2 2 0 4 1 1 4 5 3 4 5 2 2 3 3 1 2 1 4 1 4 1 5 1 5 5 3 2 1 4 3 1 1 0 3 1 1 1 1 4 4 1 1 1 2 3 4 2 2 5 0 3 1 3 3 0 5 3 4 2 4 3 5 1 5 4 5 2 1 2 1 1 4 4 1 4 3 2 3 5 4 2 4 3 1 5 1 1 1 1 3 0 1 2 2 3 1 4 3 3 2 1 4 1 2 2 2 4 3 2 2 5 1 1 5 1 4 1 2 1 2 1 5 5 2 1 1 4 2 1 5 3 2 3 5 5 1 1 5 0 4 3 5 1 1 2 4 4 3 4 5 5 4 1 4 3 0 1 2 3 3 4 5 3 4 4 4 3 4 4 5 1 2 4 1 1 3 0 5 3 2 0 1 5 1 1 4 1 4 4 5 4 1 2 3 3 5 2 3 3 3 4 1 5 1 2 2 4 2 3 1 1 1 4 5 0 3 4 4 5 4 1 5 1 3 4 5 0 1 3 1 3 2 3 2 4 5 4 4 1 2 2 3 1 1 2 

In [15]:
# load the test data

TestData=pd.read_excel('testReputation.xlsx' )
TestData=TestData.iloc[:,:-2].astype(float)
TestData=TestData/200

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'testOut.tsv', sep='\t')

TestData=pd.concat([TestData,TF_Output], axis=1)


TestData=torch.tensor(TestData.values)
TestData

tensor([[ 0.0050,  0.0100,  0.0050,  ...,  0.7627,  0.9614,  0.9639],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.1152, -0.4983, -0.6050],
        [ 0.0200,  0.0250,  0.0600,  ...,  0.1076, -0.3293, -0.4626],
        ...,
        [ 0.0100,  0.0050,  0.0250,  ..., -0.4456, -1.0010, -0.7314],
        [ 0.2200,  0.0950,  0.0350,  ..., -0.0431, -0.6367, -0.5122],
        [ 0.0050,  0.0600,  0.0100,  ...,  0.0622, -0.0334,  0.0844]],
       dtype=torch.float64)

In [32]:
labels=pd.read_excel('testReputation.xlsx' )

labels=labels.iloc[:,-1] 
labelsOneHot=pd.get_dummies(labels)
labelsOneHot

,0,1,2,3,4,5
0,0,0,0,0,1,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
1278,0,1,0,0,0,0
1279,0,0,0,0,1,0
1280,0,0,1,0,0,0
1281,1,0,0,0,0,0


In [17]:
TestLables =torch.tensor(labelsOneHot.values)
TestLables

tensor([[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0],
        ...,
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=torch.uint8)

In [33]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(TestData)):
        outputs = net(TestData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if labelsOneHot.iloc[row,result]==1: correct+=1
        
        Y.append(result)
        Pred.append(labels.iloc[row])
        
        print(result, end=' ')
        
       
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))

4 3 3 4 3 3 3 4 5 4 3 3 3 1 5 3 4 1 1 4 4 4 5 3 4 3 4 3 3 5 4 5 0 3 4 4 2 3 5 5 1 1 3 1 1 3 5 2 2 5 4 4 3 1 3 3 5 5 5 1 1 4 1 3 1 4 3 4 2 5 5 1 3 4 1 1 5 1 5 3 2 1 5 4 1 5 1 2 5 4 4 1 1 1 5 3 5 5 3 2 4 0 4 1 3 1 1 2 1 5 1 3 3 4 1 0 1 4 1 2 2 2 4 3 1 1 1 4 5 1 3 3 0 1 1 2 5 1 2 3 4 3 1 3 3 4 4 3 4 1 0 3 1 4 5 3 4 5 1 3 4 1 5 1 3 5 4 4 3 1 2 3 5 5 1 3 3 2 1 2 1 4 1 1 1 0 2 1 4 4 2 5 5 0 1 1 2 2 3 1 1 4 5 3 4 1 3 5 4 3 2 3 1 0 1 1 3 4 1 1 4 1 5 3 3 5 0 4 3 0 4 1 0 1 3 2 2 0 4 2 2 1 2 1 4 4 2 1 3 3 0 5 5 4 3 3 5 0 4 1 3 1 5 3 0 3 1 4 4 2 1 3 1 2 5 2 5 1 5 4 5 1 3 5 1 5 3 1 5 0 3 4 0 1 1 4 3 5 3 4 1 1 5 5 5 3 0 5 3 4 1 4 1 3 3 4 5 3 3 3 3 4 5 4 1 0 5 1 5 3 4 3 3 2 5 1 1 1 4 2 4 2 5 1 3 5 5 3 5 2 1 0 4 1 4 1 1 4 1 2 4 1 1 1 2 0 4 2 3 1 0 1 5 5 4 3 4 5 3 1 3 1 3 4 1 3 3 3 3 3 3 2 1 1 2 2 3 3 3 4 3 3 0 1 4 5 2 5 3 3 1 2 1 3 1 1 1 4 3 4 2 4 4 1 1 3 4 3 4 3 5 4 2 5 2 4 3 1 2 1 5 5 1 1 3 1 3 3 1 5 1 3 1 3 4 3 3 3 3 1 5 2 3 4 5 2 1 3 1 3 0 4 1 4 3 2 3 1 3 3 2 3 3 1 2 5 2 0 1 5 1 5 4 3 2 0 3 4 0 4 

In [17]:
from sklearn import metrics
 
print(metrics.confusion_matrix(Y,Pred))

[[ 50  10   9   6   1   3]
 [ 17 145  35  48  20  25]
 [  6  26  89  30  20  12]
 [ 12  31  40 112  52  26]
 [  3  25  24  47 126  44]
 [  4  13  17  24  30 101]]


In [33]:
target_names = ['Pants', 'False', 'Barely-True','Hlaf-True','Mostly-True','True']

print(metrics.classification_report(Y, Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.65      0.47      0.54       116
       False       0.48      0.53      0.51       263
 Barely-True       0.50      0.45      0.47       237
   Hlaf-True       0.48      0.51      0.49       248
 Mostly-True       0.49      0.51      0.50       251
        True       0.43      0.44      0.43       169

    accuracy                           0.49      1284
   macro avg       0.50      0.48      0.49      1284
weighted avg       0.49      0.49      0.49      1284

